In [1]:
!cd

C:\Users\admin\Desktop\데이터 공부 자료\day20\2021-K-Digital-Training-main\SQL


In [2]:
import pymysql.cursors
# Connect to the database

In [3]:
import pandas as pd

In [4]:
import pandas_profiling

In [5]:
df = pd.read_csv('tips.csv')

In [6]:
df=df.assign(tip_rate=((df["tip"] / df["total_bill"])*100).round(2))

In [7]:
df['sex'].replace({'Female':0,'Male':1},inplace=True)

In [8]:
df['smoker'].replace({'No':0,'Yes':1},inplace=True)

In [9]:
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3},inplace=True)

In [10]:
df['time'].replace({'Lunch':0,'Dinner':1},inplace=True)

In [ ]:
df.isnull().sum()         #data frame에서 nan 찾기

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
tip_rate      0
dtype: int64

In [ ]:
df.describe()

In [ ]:
type(df)

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
data=df.values           #data frame -> array로 바꾸기

In [ ]:
y=df['tip']    #구하고자 하는 값

In [ ]:
X=df.drop('tip',axis=1) #구하고자 하는 값 제외한 것은 모두 넣고 비교

In [ ]:
X.describe()

In [ ]:
y.describe()

In [ ]:
data=X.values

In [ ]:
y=y.values

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data[:,-1]

In [ ]:
sum(np.isnan(data).flatten())          #array에서 nan값 찾기

In [ ]:
imputer = SimpleImputer(strategy='median')

In [ ]:
imputer.fit(data)

In [ ]:
datatrans = imputer.transform(data)

In [ ]:
datatrans[:,-1]

In [ ]:
df_trans=pd.DataFrame(datatrans)

In [ ]:
df_trans.isnull()

In [ ]:
df_trans.isnull().sum()

# 시작

### Scaling Standardization
### RFE

In [ ]:
from sklearn.feature_selection import RFE
#from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
data=df_trans.values

###### df_trans는 tip의 값을 뺀 값

In [ ]:
data_sc=sc.fit_transform(data)

In [ ]:
data_sc

In [ ]:
df_sc=pd.DataFrame(data_sc,columns=['total_bill','sex','smoker','day','time','size','tip_rate'])

In [ ]:
from sklearn.svm import SVR

In [ ]:
estimator=SVR(kernel="linear")

In [ ]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4) #변수 7개 중 4개 가지기

In [ ]:
y.shape

In [ ]:
# fit RFE
selecto=rfe.fit(data_sc, y)

In [ ]:
selecto.support_

In [ ]:
df_sc.head(3)   #1,2,3이 False 즉, sex와 smoker,day가 빠졌다.

In [ ]:
,# summarize all features
for i in range(df_sc.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
rselecto= rfe.fit_transform(data_sc, y)

In [ ]:
Selecto=pd.DataFrame(rselecto)

In [ ]:
Selecto.columns=['total_bill', 'time', 'size', 'tip_rate']

In [ ]:
Selecto.describe()

In [ ]:
X.describe()

###### Selecto.describe().round()을 Standard 하지 않은 X.describe()와 비교해보자.

In [ ]:
Selecto.profile_report()

#### HW_with 의 RFE와 비교해보자. 
#### Standard 과정을 거치지 않은 python_db_Org의 RFE는 1,3,4이 False 즉 sex, day,time가 빠졌다. 
#### Standard 과정을 거친 python_db_Standardization은 1,2,3이 False 즉, sex와 smoker,day가 빠졌다.
#### 참고로 Standard->RFE가 더 바람직하다.

## 

### Scaling Normalization
### RFE

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# define the scaler
trans = MinMaxScaler()

In [ ]:
# transform the data
data_norm = trans.fit_transform(data)

In [ ]:
df_norm=pd.DataFrame(data_norm,columns=['total_bill','sex','smoker','day','time','size','tip_rate'])

In [ ]:
df_norm.describe()

In [ ]:
X.describe()

###### 기존의 데이터인 X와 비교를 해보자.

In [ ]:
estimator=SVR(kernel="linear")

In [ ]:
# define RFE
rfe = RFE(estimator, n_features_to_select=4) #변수 7개 중 4개 가지기

In [ ]:
# fit RFE
selecto=rfe.fit(data_norm, y)

In [ ]:
selecto.support_

In [ ]:
df_norm.head(3)

In [ ]:
,# summarize all features
for i in range(df_norm.shape[1]):
  print('Column: %d, Selected=%s, Rank: %d' % (i, rfe.support_[i], rfe.ranking_[i]))

In [ ]:
nselecto= rfe.fit_transform(data_norm, y)

In [ ]:
Nselecto=pd.DataFrame(nselecto)

In [ ]:
Nselecto.columns=['total_bill', 'smoker', 'size', 'tip_rate']

In [ ]:
Nselecto.describe()

In [ ]:
X.describe()

In [ ]:
Nselecto.profile_report()

#### HW_with 의 RFE,  Standard 과정과 비교하자.
#### Standard 과정을 거치지 않은 python_db_Org의 RFE는 1,3,4이 False 즉 sex, day,time가 빠졌다. 
#### Standard 과정을 거친 python_db_Standardization은 1,2,3이 False 즉, sex와 smoker,day가 빠졌다.
#### Norm 과정을 거친 python_db_Standardization은 1,3,4이 False 즉 sex, day,time가 빠졌다.
#### 참고로 Standard->RFE 혹은 Norm->RFE가 더 바람직하다. 
#### 파생변수인 tip_rate는 세 과정 중 빠지는 경우가 없으므로 매우 잘 설정한 변수이다.

# MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
trans = MinMaxScaler()

In [ ]:
data.shape

In [ ]:
X_norm = trans.fit_transform(data)

In [ ]:
df_norm=pd.DataFrame(X_norm)

In [ ]:
df_norm.describe().round()

# Data Standardization

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc = StandardScaler()

In [ ]:
df_sc=sc.fit_transform(data)

In [ ]:
df_trsfrom_sc=pd.DataFrame(df_sc)

In [ ]:
df_trsfrom_sc.describe().round() #정규화가 끝나서 평균0, 분산1

# PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# define the transform
trans = PCA(n_components=4)

In [ ]:
# transform the data
X_dim = trans.fit_transform(datatrans)

In [ ]:
# summarize data after the transform
print(X_dim[:3, :]) #X_dim은 array라서 describe(),info() 불가

In [ ]:
x_dim=pd.DataFrame(X_dim)

In [ ]:
x_dim.describe().round()

# Regression Feature Selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [ ]:
# define feature selection
fs = SelectKBest(score_func=f_regression, k=4)

In [ ]:
datatrans.shape

In [ ]:
y.shape

In [ ]:
# apply feature selection / 행이 같으므로 값이 나온다.
X_selected = fs.fit_transform(datatrans, y)
print(X_selected.shape)

#### rfe는 독립변수들과 목표변수와의 관계를 통해 차원을 축소시키고
#### pca는 독립변수들끼리의 상관관계만으로만 차원을 축소시킨다.

In [ ]:
#예전에 공부했던 내용들

In [ ]:
df.total_bill

In [ ]:
df['total_bill']

In [ ]:
df['total_bill'].hist()

In [ ]:
#새출발

In [ ]:
import pymysql.cursors
import pandas as pd

In [ ]:
sql="select customers.customerName, payments.checkNumber from customers left join payments on customers.customerNumber=payments.customerNumber where payments.paymentDate >= '2004-10-06';"

In [ ]:
conn = pymysql.connect(host='localhost', user='me', 
                       password='9421', db='classicmodels', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

   with conn.cursor() as curs:
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)
      print(df)
    
    #df.to_csv('query.csv')

finally:

   conn.close()

In [ ]:
df.to_csv('query_car.csv')